# RAG using Meta AI Llama-3


<img src="./resources/rag_architecture.png" width=800px>

In [1]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# add your documents in this directory, you can drag & drop
input_dir_path = '/teamspace/studios/this_studio/medical_documents'

In [4]:
# setup llm & embedding model
llm=Ollama(model="llama3:8b-instruct-q4_1", request_timeout=120.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Load data from PDF files
loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== Customize prompt template for clinical assistance ======
broad_prompt = '''
You are an expert clinical assistant AI. Your task is to help healthcare providers with diagnostic support based on patient symptoms, medical history, and local disease data.

- If the patient's symptoms match a known diagnosis based on your pre-existing knowledge or provided documents, suggest the diagnosis and treatment options.
- Otherwise, use the search tool to retrieve relevant medical information.
- In addition to diagnosis, suggest follow-up consultation questions for the healthcare provider to ask the patient to ensure a comprehensive diagnosis.
- Provide classifications of possible diseases based on the symptoms, including differential diagnoses (similar conditions to rule out).
- Ensure to ask further clarifying questions that the doctor should consider, such as lifestyle factors, prior medical history, or exposure to local disease trends, to refine the diagnosis.
- If the user does not provide all the necessary information (e.g., symptoms, medical history, or location), ask them to specify the missing fields.
'''

# Create the prompt template
qa_prompt_tmpl = PromptTemplate(broad_prompt)

# Update the query engine with the new prompt template
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response with an example query
response = query_engine.query("What are the potential diagnoses for a patient with a cough, fever, and asthma?",)

# Print the generated response
print("Generated Response:", response)


Parsing nodes:   0%|          | 0/5229 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1673 [00:00<?, ?it/s]

Generated Response: I'm happy to assist healthcare providers with diagnostic support. To get started, I need some information about the patient's symptoms and medical history. Please provide me with the following details:

* Patient's symptoms (please describe in as much detail as possible)
* Medical history (including any previous diagnoses or conditions)
* Location of the patient (city, state/province, country)

With this information, I'll do my best to suggest a diagnosis and treatment options. If necessary, I'll also provide follow-up consultation questions for you to ask the patient.

Please note that I'll need all the necessary information to provide an accurate diagnosis. If there's any missing data, I'll let you know and ask you to fill in the gaps.

Let's get started!


In [7]:
display(Markdown(str(response)))

I'm happy to assist healthcare providers with diagnostic support. To get started, I need some information about the patient's symptoms and medical history. Please provide me with the following details:

* Patient's symptoms (please describe in as much detail as possible)
* Medical history (including any previous diagnoses or conditions)
* Location of the patient (city, state/province, country)

With this information, I'll do my best to suggest a diagnosis and treatment options. If necessary, I'll also provide follow-up consultation questions for you to ask the patient.

Please note that I'll need all the necessary information to provide an accurate diagnosis. If there's any missing data, I'll let you know and ask you to fill in the gaps.

Let's get started!

### ❗️❗️ Make sure you clear GPU memory by clicking on Restart button above, if you want to use Streamlit from here

In [1]:
# check GPU usage

!nvidia-smi

Mon Sep 23 19:01:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P8              21W / 300W |      3MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--